In [1]:
import sys
sys.path.insert(0, '../..')  # Adjust path if needed
import omama as O
import time

In [2]:
# Load Omama data loader and cache
omama_loader = O.OmamaLoader()
data = O.Data(data_loader=omama_loader, load_cache=True)

Loading config data from ini file
DataLoader type is:  <class 'omama.loaders.omama_loader.OmamaLoader'>


In [3]:
img1 = O.DataHelper.get(image='DXm.2.25.166394588784602078462152973218093905294')  
img2 = O.DataHelper.get(image='DXm.2.25.211728782949764576548869143846819217941')  
img3 = O.DataHelper.get(image='DXm.2.25.301398532875079675491124913846193631972')  
img4 = O.DataHelper.get(image='DXm.2.25.74245724832122045242589684382329849414')

In [4]:
pred_api = O.DeepSight.run([img1, img2, img3, img4], timing=True)

Time to parse SOP UIDs:  9.059906005859375e-06
Time to check cache:  0.9847567081451416
Running DeepSight on 2 cases, please be patient...
Time to update cache:  3.1665866374969482
...took 60.85214924812317


In [5]:
for img in [img1, img2, img3, img4]:
    sop_uid = img.SOPInstanceUID
    if sop_uid in pred_api:
        print(f"Showing API Prediction for {sop_uid}")
        O.DataHelper.view(image=img, roi_coords=pred_api[sop_uid]['coords'])
    else:
        print(f"No API Prediction for {sop_uid}")

Showing API Prediction for 2.25.166394588784602078462152973218093905294
type of data:  <class 'omama.data.Data'>
bb: [1780, 1646, 1995, 1902]
Showing API Prediction for 2.25.211728782949764576548869143846819217941
type of data:  <class 'omama.data.Data'>
bb: [1811, 1177, 1935, 1309]
Showing API Prediction for 2.25.301398532875079675491124913846193631972
type of data:  <class 'omama.data.Data'>
bb: [175, 884, 302, 1004]
Showing API Prediction for 2.25.74245724832122045242589684382329849414
type of data:  <class 'omama.data.Data'>
bb: [28, 543, 253, 832]


In [6]:
import json

with open("/raid/mpsych/OMAMA/2025/PLAYGROUND/out5/2.25.53854178431193012953122685308093257260/results_full.json", "r") as f:
    pred_shell = json.load(f)

In [7]:
for img in [img1, img2, img3, img4]:
    sop = img.SOPInstanceUID
    api_score = pred_api[sop]['score'] if sop in pred_api else 'N/A'
    
    shell_score = 'N/A'
    if 'results' in pred_shell and 'dicom_results' in pred_shell['results']:
        shell_data = pred_shell['results']['dicom_results'].get(sop)
        if shell_data:
            shell_score = shell_data['none'][0]['score']
    
    print(f"SOP: {sop}")
    print(f"  ➤ API Score:   {api_score}")
    print(f"  ➤ Shell Score: {shell_score}")
    print("----------")

SOP: 2.25.166394588784602078462152973218093905294
  ➤ API Score:   0.09978759288787842
  ➤ Shell Score: 0.09978759288787842
----------
SOP: 2.25.211728782949764576548869143846819217941
  ➤ API Score:   0.8241342306137085
  ➤ Shell Score: 0.8241342306137085
----------
SOP: 2.25.301398532875079675491124913846193631972
  ➤ API Score:   0.341439813375473
  ➤ Shell Score: 0.341439813375473
----------
SOP: 2.25.74245724832122045242589684382329849414
  ➤ API Score:   0.11451892554759979
  ➤ Shell Score: 0.11451892554759979
----------


In [8]:
study_results = pred_shell['results']['study_results']
print("Shell Study-Level Scores:")
print(f"  ➤ Left Breast (L): {study_results['L']}")
print(f"  ➤ Right Breast (R): {study_results['R']}")
print(f"  ➤ Total: {study_results['total']}")

Shell Study-Level Scores:
  ➤ Left Breast (L): {'score': 0.2279793694615364, 'category': 0, 'category_name': ''}
  ➤ Right Breast (R): {'score': 0.46196091175079346, 'category': 1, 'category_name': 'Suspicious'}
  ➤ Total: {'score': 0.46196091175079346, 'category': 1, 'category_name': 'Suspicious', 'postprocessed_percentile_score': 89.31568000086168}
